# 🤔Movie lens 내쉬균형을 이용해서 어쩌구

### context

1. Data load
2. SVD 평점 예측
3. Clustering
4. NASH aggregation
5. Measure

## 데이터 LOAD

## Library 

In [5]:
#기본 라이브러리
import pandas as pd
import numpy as np
import math
import time

# SVD 행렬축소
from sklearn.decomposition import TruncatedSVD

# 제작 함수
import HA_ndcg

In [215]:

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')

#컬럼명 수정
ratings.columns = ['userid','movieid','rating','timestamp']

#시간 컬럼 제거
ratings = ratings.drop(['timestamp'], axis=1)

print("총 주어진 ratings 데이터 : ", ratings.shape)

#피벗테이블로 만들어서 사용자-아이템 행렬 확인하기
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])
rating_table

총 주어진 ratings 데이터 :  (100000, 3)


movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Trian 데이터와 Test 데이터는 10만개의 데이터를 8만개와 2만개로 나누어 분류했다.  
이후 pivot 하여 User-item matrix를 만들었다.

In [218]:
# 80000 row
user_train = pd.read_csv('../ml-100k_GRS/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]
user_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

#nan 값 그대로 받고 싶을때 
#user_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

# 20000 row
user_test = pd.read_csv('../ml-100k_GRS/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]
user_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

In [219]:
user_train

itemID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. MF 로 평점 예측

내쉬균형을 도출하기 위해 우선 모든 영화들에 대한 예측평점을 알고 있다고 가정했다.  
수정 필요

In [220]:
## 재형이형이 준 코드

import numpy

def matrix_factorization(R, P, Q, K, steps=300, alpha=0.0002, beta=0.02):

    

    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter
    '''
    Q = Q.T

    start = time.time()
    for step in range(steps):
        print('epoch: %d, time: %f'%(step, time.time()-start))
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[0])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [175]:
# N: num of User
user_train_np = user_train.to_numpy()

N = len(user_train_np)
# M: num of Movie
M = len(user_train_np[0])

K = 5  #이거를 [5, 10, 15, 20, 25, 30]
 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(user_train_np, P, Q, K)

epoch: 0, time: 0.000989
epoch: 1, time: 3.280353
epoch: 2, time: 6.285608
epoch: 3, time: 9.324106
epoch: 4, time: 12.248699
epoch: 5, time: 15.411378
epoch: 6, time: 18.371295
epoch: 7, time: 21.719843
epoch: 8, time: 25.065463
epoch: 9, time: 28.902917
epoch: 10, time: 32.917323
epoch: 11, time: 36.577510
epoch: 12, time: 39.932499
epoch: 13, time: 43.038652
epoch: 14, time: 46.481521
epoch: 15, time: 49.451307
epoch: 16, time: 52.422859
epoch: 17, time: 55.378347
epoch: 18, time: 58.357146
epoch: 19, time: 61.294735
epoch: 20, time: 64.226367
epoch: 21, time: 67.193288
epoch: 22, time: 70.126239
epoch: 23, time: 73.100387
epoch: 24, time: 76.129144
epoch: 25, time: 79.099834
epoch: 26, time: 82.111113
epoch: 27, time: 85.402485
epoch: 28, time: 88.428380
epoch: 29, time: 91.527903
epoch: 30, time: 94.536308
epoch: 31, time: 97.450888
epoch: 32, time: 100.428622
epoch: 33, time: 103.794512
epoch: 34, time: 106.684748
epoch: 35, time: 109.974392
epoch: 36, time: 113.025990
epoch: 37,

In [221]:
Ratings = pd.DataFrame(nP@nQ.T)
Ratings

,0,1,2,3,4,5,6,7,8,9,...,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649
0,4.080783,3.105075,3.802179,3.739923,3.404238,3.538765,3.872827,3.764835,4.283478,3.862410,...,2.080672,2.153241,3.143997,2.510603,2.744466,2.334731,2.995038,2.459726,1.780943,3.224237
1,4.139741,3.401644,3.009595,3.671248,3.488930,3.721358,3.883205,4.162529,3.994967,3.913301,...,2.522858,2.512630,2.847540,2.569968,2.578417,2.585012,3.086535,2.435236,2.194503,3.009415
2,3.812366,2.541412,2.631012,2.892696,3.049660,2.968664,3.263037,3.445256,3.679177,3.049477,...,2.087686,1.903964,2.421638,2.296500,2.525421,1.966207,2.509840,2.288322,1.648895,2.673915
3,4.365309,4.104658,3.656998,4.355253,4.028333,4.991767,5.044913,5.039673,4.485047,5.312385,...,3.194719,3.571967,3.316243,3.088087,2.814687,3.291098,3.835163,2.587822,3.023613,3.555578
4,3.593027,3.539493,2.962253,3.827104,3.081620,2.906670,2.953503,3.463096,3.261872,3.280941,...,1.787562,1.750811,2.820216,1.849225,1.964371,2.367268,2.675956,1.920003,1.661421,2.538068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,5.207394,4.426681,3.615326,4.655460,4.575027,4.696923,4.812633,5.291031,4.750219,4.965793,...,3.236877,3.173927,3.555299,3.113396,3.233136,3.323435,3.848536,2.885239,2.884222,3.684983
939,3.438722,3.015706,2.437885,3.115329,3.179988,3.548306,3.581239,3.767293,3.205392,3.729757,...,2.426728,2.513905,2.375810,2.239195,2.224985,2.383906,2.736020,1.907813,2.236562,2.557144
940,4.399331,3.475677,3.630721,3.968373,3.693083,3.723816,3.988878,4.138629,4.339818,4.016469,...,2.353886,2.330675,3.210518,2.613850,2.830984,2.572275,3.169721,2.563716,2.023777,3.277864
941,4.270895,3.638857,3.098937,3.839040,3.846248,4.160875,4.254241,4.493551,4.009143,4.391656,...,2.819349,2.872302,2.966109,2.699632,2.748011,2.825610,3.290355,2.392754,2.554145,3.148578


---

## 3. 클러스터링

In [222]:
#km.cluster_centers_ # 중심

array([[4.27339567, 3.59333016, 3.24472791, ..., 2.50936133, 2.39879788,
        3.19693521],
       [3.40337601, 2.79195794, 2.56368457, ..., 2.04872495, 1.85075671,
        2.54156053],
       [4.00094382, 3.36661473, 2.99669516, ..., 2.35208106, 2.20540903,
        2.96304706],
       [3.70429795, 3.08096173, 2.86400895, ..., 2.21881775, 2.03530231,
        2.79475112],
       [2.84888137, 2.35340348, 2.14535653, ..., 1.63986121, 1.55669138,
        2.10438254],
       [4.78491655, 4.07003593, 3.54127733, ..., 2.76778417, 2.70865967,
        3.5312355 ]])

In [247]:
from sklearn.cluster import KMeans

# SVD 기반 클러스터링 한 애들을 pure 한 train, test set 에 적용
def clustering(num, SVD_ratings, train_data, test_data):
    
    # 1. clusturing
    km = KMeans(n_clusters=num, init='k-means++')
    cluster = km.fit(SVD_ratings)
    cluster_id = pd.DataFrame(cluster.labels_)                   # 모든  user에 클러스터 n 이 표시됨

    cluster_id.index = SVD_ratings.index                        # userId 칼럼을 인덱스로 설정 1~943
    cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터


    ## 2. train, test 에 cluster 정보 추가
    train_data = pd.concat([train_data, cluster_id], axis=1, join='inner')
    test_data = pd.concat([test_data, cluster_id], axis=1, join='inner')

    km_center = km.cluster_centers_ # 중심
    #print(km.feature_names_in_)
    return train_data, test_data, km_center


In [248]:
train_data, test_data, km_center = clustering(6, Ratings, user_train, user_test)

#각 클러스터 당 인원수
train_data['cluster'].value_counts()

5    270
0    229
4    197
3    139
2     65
1     42
Name: cluster, dtype: int64

In [279]:
round(Ratings[0],4)

0      4.0808
1      4.1397
2      3.8124
3      4.3653
4      3.5930
        ...  
938    5.2074
939    3.4387
940    4.3993
941    4.2709
942    3.9138
Name: 0, Length: 943, dtype: float64

In [266]:
Ratings[Ratings == km_center[0][0]].notnull().sum().sum()

0

In [253]:
Ratings.iloc[:,:][Ratings.iloc[:,:] == km_center[0]].dropna(axis=0, how='all')

,0,1,2,3,4,5,6,7,8,9,...,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649


In [269]:
km_center.dtype

dtype('float64')

# 1. Average 를 통한 추천

In [189]:
# cluster 별 영화 별점 평균 행렬값 (index : cluster)
def cluster_mean(train_data):
    #train 데이터를 클러스터링으로 index 처리
    
    mean_rating = pd.DataFrame(columns = train_data.columns)
    mean_rating.set_index('cluster')
    mean_rating

    num = 6

    # 클러스터 별 사용자들의 영화 평균 별점 도출 (아무도 보지 않은 것은 Nan값 처리 된다.)
    for i in range(num):
        mean_rating = mean_rating.append(train_data[train_data.cluster == i].mean(axis=0), ignore_index=True) 

    return mean_rating.drop(['cluster'], axis=1)

mean_rating = cluster_mean(train_data)

In [190]:
# train 데이터와 test 데이터에서 겹치지 않는 아이템(영화) 를 제거한다.

def miss_matching(mean_data, test_data):

    for c in mean_data.columns:
        if c not in test_data.columns:
            del mean_data[c]
        
    for c in test_data.columns[:-1]:
        if c not in mean_data.columns:
            del test_data[c]

    return mean_data, test_data


pred_data, test_data =  miss_matching(mean_rating,test_data)

In [191]:
# top k 를 뽑을 때
pred_data[0:1].sort_values(by=0,axis=1, ascending=False)

,50,100,181,127,174,1,286,258,288,98,...,1242,1398,1250,1252,1390,1386,1385,1379,1378,1094
0,2.235507,1.768116,1.757246,1.65942,1.57971,1.554348,1.492754,1.478261,1.471014,1.438406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
print(f'예측된 클러스터링 별 item 평점과 예측할 test item은 \n{pred_data.shape}, {test_data.shape} 로 동일해졌다.')

예측된 클러스터링 별 item 평점과 예측할 test item은 
(6, 1378), (459, 1379) 로 동일해졌다.


In [59]:
#ndcg 값을 구하기 위해 추천되지 않은 아이템들은 0점을 준다.

pred_data = pred_data.fillna(0, downcast='infer')
test_data = test_data.fillna(0, downcast='infer')

- test 데이터의 마지막 열은 clustering number 을 의미한다

- 칼럼은 1682 까지 있지만 아무 클러스터도 보지 않았던 행은 자동으로 제거된다.

해당 데이터를 통해 클러스터마다 가장 높은 별점 top 5를 추천해준다.


In [193]:
from sklearn.metrics import ndcg_score

def get_NDCG(pred_data, test_data, num=6):

    #각 클러스터 별 결과
    result = [0]*num

    #각 클러스터 별 인원수
    length = list(test_data.cluster.value_counts().sort_index())

    for idx in test_data.index:

        # -1 해당 user 의 그룹을 확인
        cluster_num = int(test_data.loc[idx].cluster) 
        
        # -2 train 에서의 예측치와 test 데이터 간 각 user 별 평점 NDCG 값 계산
        result[cluster_num] += ndcg_score([test_data.loc[idx][:-1]], [pred_data.loc[cluster_num]], k=300) #  k=10

        # ndcg 는 상위 k의 것을 대상으로 점수를 매길 수 있다.
        #result[cluster] += ndcg_score([user_item_test_cl.loc[idx][:-1]], [mean_rating.loc[cluster]], k=4)

    ## 5. 최종적으로 각 nDCG값 / 각 cluster의 요소 개수
    for i in range(6):
        # 클러스터 별 유저 NDCG 의 평균
        result[i] = result[i]/length[i]

    print(f"cluster 수 : {len(length)}\ncluster 별 인원 수 : {length}")
    print(f"총 NDCG : {(sum(result)/len(length)):.4f} \n\n ")

get_NDCG(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [131, 32, 62, 126, 90, 18]
총 NDCG : 0.3671 

 


In [196]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# precision & recall
def get_pre_rec(pred_data, test_data):


    # 1. 전처리
    # 평점이 2.5 이상이면 정답, 아닌 것은 오답으로 평가하였다.
    pred_data[pred_data < 2.5]= 0 #원래는 4 이상으로 측정
    pred_data[pred_data >= 2.5]= 1

    # test 데이터 열에는 cluster 정보가 있기 때문에 백업하는 작업을 거친다.
    test_data_cluster = test_data[['cluster']].copy()
    test_data[test_data < 2.5] = 0
    test_data[test_data >= 2.5] = 1
    test_data[['cluster']] = test_data_cluster


    # 2. pre rec 계산
    pre_result = [0]*6 # 결과값 저장 리스트
    rec_result = [0]*6

    length = list(test_data.cluster.value_counts().sort_index())

    ## 4. 각 결과 값에 precision, recall더해줌
    for idx in test_data.index:
        cluster_num = int(test_data.loc[idx].cluster)
        pre_result[cluster_num] += precision_score(list(test_data.loc[idx][:-1].values) ,list(pred_data.loc[cluster_num].values ), average = 'binary')
        rec_result[cluster_num] += recall_score(list(test_data.loc[idx][:-1].values) ,list(pred_data.loc[cluster_num].values ), average = 'binary')

    for i in range(6):
        # 클러스터 별 유저 PRE/REC 의 평균
        pre_result[i] = pre_result[i]/length[i]
        rec_result[i] = rec_result[i]/length[i]

    print(f"cluster 수 : {len(length)}\ncluster 별 인원 수 : {length}")
    print(f"Presicion : {(sum(pre_result)/len(length)):.4f}")
    print(f"recall : {(sum(rec_result)/len(length)):.4f}")

In [197]:
get_pre_rec(pred_data, test_data)


cluster 수 : 6
cluster 별 인원 수 : [131, 32, 62, 126, 90, 18]
Presicion : 0.0000
recall : 0.0000


## 2. Least Misery 를 통한 추천

In [198]:
train_data, test_data = clustering(6, Ratings, user_train, user_test)

#각 클러스터 당 인원수
train_data['cluster'].value_counts().sort_index()

0     66
1    136
2    199
3     42
4    236
5    263
Name: cluster, dtype: int64

In [199]:
def cluster_LM(train_data):
    #train 데이터를 클러스터링으로 index 처리
    
    mean_rating = pd.DataFrame(columns = train_data.columns)
    mean_rating.set_index('cluster')
    mean_rating

    num = 6

    # 클러스터 별 사용자들의 영화 평균 별점 도출 (아무도 보지 않은 것은 Nan값 처리 된다.)
    for i in range(num):
        mean_rating = mean_rating.append(train_data[train_data.cluster == i].min(axis=0), ignore_index=True) 

    return mean_rating.drop(['cluster'], axis=1)

LM_rating = cluster_LM(train_data)

In [200]:
pred_data, test_data =  miss_matching(LM_rating,test_data)

In [201]:
pred_data = pred_data.fillna(0)
test_data = test_data.fillna(0)

In [202]:
get_NDCG(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [35, 61, 98, 18, 125, 122]
총 NDCG : 0.1016 

 


LM 방식으로 했을 때 NDCG 값이 약 0.2 만큼 떨어진다.

In [203]:
get_pre_rec(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [35, 61, 98, 18, 125, 122]
Presicion : 0.0000
recall : 0.0000


---

In [208]:
user_train #nan 값으로 채워진 train 데이터

itemID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
Ratings #  MF 로 모든 별점을 채운 버전

,0,1,2,3,4,5,6,7,8,9,...,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649
0,4.080783,3.105075,3.802179,3.739923,3.404238,3.538765,3.872827,3.764835,4.283478,3.862410,...,2.080672,2.153241,3.143997,2.510603,2.744466,2.334731,2.995038,2.459726,1.780943,3.224237
1,4.139741,3.401644,3.009595,3.671248,3.488930,3.721358,3.883205,4.162529,3.994967,3.913301,...,2.522858,2.512630,2.847540,2.569968,2.578417,2.585012,3.086535,2.435236,2.194503,3.009415
2,3.812366,2.541412,2.631012,2.892696,3.049660,2.968664,3.263037,3.445256,3.679177,3.049477,...,2.087686,1.903964,2.421638,2.296500,2.525421,1.966207,2.509840,2.288322,1.648895,2.673915
3,4.365309,4.104658,3.656998,4.355253,4.028333,4.991767,5.044913,5.039673,4.485047,5.312385,...,3.194719,3.571967,3.316243,3.088087,2.814687,3.291098,3.835163,2.587822,3.023613,3.555578
4,3.593027,3.539493,2.962253,3.827104,3.081620,2.906670,2.953503,3.463096,3.261872,3.280941,...,1.787562,1.750811,2.820216,1.849225,1.964371,2.367268,2.675956,1.920003,1.661421,2.538068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,5.207394,4.426681,3.615326,4.655460,4.575027,4.696923,4.812633,5.291031,4.750219,4.965793,...,3.236877,3.173927,3.555299,3.113396,3.233136,3.323435,3.848536,2.885239,2.884222,3.684983
939,3.438722,3.015706,2.437885,3.115329,3.179988,3.548306,3.581239,3.767293,3.205392,3.729757,...,2.426728,2.513905,2.375810,2.239195,2.224985,2.383906,2.736020,1.907813,2.236562,2.557144
940,4.399331,3.475677,3.630721,3.968373,3.693083,3.723816,3.988878,4.138629,4.339818,4.016469,...,2.353886,2.330675,3.210518,2.613850,2.830984,2.572275,3.169721,2.563716,2.023777,3.277864
941,4.270895,3.638857,3.098937,3.839040,3.846248,4.160875,4.254241,4.493551,4.009143,4.391656,...,2.819349,2.872302,2.966109,2.699632,2.748011,2.825610,3.290355,2.392754,2.554145,3.148578


In [210]:
train_data # cluster 가 표기되어 있는 train 데이터

,1,2,3,4,5,6,7,8,9,10,...,1674,1675,1676,1677,1678,1679,1680,1681,1682,cluster
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [211]:
test_data #테스트 데이터

,1,2,3,4,5,6,7,8,9,10,...,1530,1531,1534,1538,1550,1554,1555,1578,1591,cluster
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [213]:
km_center

NameError: name 'km_center' is not defined

<h2>3. 내쉬 균형을 이용한 추천</h2>

In [ ]:
import nashpy as nash

ntrainData = train_data
